- Object Classification


In [ ]:
# On a utiliser python 3.7

# Installer Keras
!python -m pip install Keras==2.2.4 Keras-Applications==1.0.8 Keras-Preprocessing==1.1.2

# Installer TensorFlow
!python -m pip install tensorflow==1.15.0 tensorflow-estimator==1.15.1 tensorboard==1.15.0

# Installer Scikit-Image (en installant également ses dépendances courantes)
!python -m pip install scikit-image==0.16.2

# Autres dépendances essentielles
!python -m pip install numpy==1.21.6 scipy==1.7.3 pandas==1.0.3 matplotlib==3.5.3 Pillow==9.5.0

# Packages supplémentaires pour la manipulation et l'affichage des données
!python -m pip install seaborn==0.11.2 tqdm==4.66.6 requests==2.31.0

# Pour la manipulation JSON et les fichiers de labels (Labelme2COCO, Pybboxes)
!python -m pip install jsonschema==4.17.3 labelme2coco==0.2.6 pybboxes==0.1.6

# Installations pour des modules auxiliaires comme l'interface utilisateur et le client Jupyter
!python -m pip install jupyter-client==7.4.9 jupyter-core==4.12.0 nest-asyncio==1.6.0

# Packages pour le développement d'algorithmes et calculs mathématiques avancés
!python -m pip install absl-py==2.1.0 cloudpickle==2.2.1 dask==2022.2.0

# Packages de manipulation d'images et de fichiers multimédia
!python -m pip install opencv-python==3.4.13.47 imageio==2.10.4

# Installations pour les réseaux de neurones et autres applications IA
!python -m pip install torch==1.13.1 sahi==0.11.18

# Installations pour gérer les fichiers et le système
!python -m pip install fsspec==2023.1.0 importlib-resources==5.12.0 importlib-metadata==6.7.0

# Installations pour la manipulation de JSON et la gestion des chemins de fichiers
!python -m pip install attrs==24.2.0 pyparsing==3.1.4 typing-extensions==4.7.1

# Gestion des demandes réseau et de l'interface utilisateur
!python -m pip install urllib3==1.26.16 termcolor==2.3.0 colorama==0.4.6

# Autres utilitaires importants pour le système et les structures de données
!python -m pip install setuptools==68.0.0 wheel==0.42.0 wrapt==1.16.0

In [ ]:
import os
import json
import numpy as np
import imgaug
import tensorflow as tf
from mrcnn.config import Config
from mrcnn import model as modellib, utils
from mrcnn.model import MaskRCNN
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

# Configuration personnalisée pour Mask R-CNN
class FlowerConfig(Config):
    NAME = "flower_tulip"
    IMAGES_PER_GPU = 4
    NUM_CLASSES = 1 + 2  # Background + Tulipe, Non-Tulipe
    STEPS_PER_EPOCH = 40
    DETECTION_MIN_CONFIDENCE = 0.8
    LEARNING_RATE = 0.001  


# Préparer le dataset
class FlowerDataset(utils.Dataset):
    def load_flowers(self, dataset_dir, subset):
        self.add_class("flower", 1, "tulipe")
        self.add_class("flower", 2, "non_tulipe")

        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        for filename in os.listdir(dataset_dir):
            if filename.endswith(".json"):
                json_path = os.path.join(dataset_dir, filename)
                with open(json_path) as f:
                    annotations = json.load(f)
                    polygons = [shape['points'] for shape in annotations['shapes']]
                    label_name = annotations['shapes'][0]['label']
                    label_id = 1 if label_name == "tulipe" else 2
                    
                    # Recherche du fichier image avec différentes extensions
                    image_name = annotations['imagePath']
                    image_path = None
                    for ext in ['.png', '.jpg', '.jpeg']:
                        potential_path = os.path.join(dataset_dir, os.path.splitext(image_name)[0] + ext)
                        if os.path.exists(potential_path):
                            image_path = potential_path
                            break
                    
                    if image_path is None:
                        print(f"Image {image_name} non trouvée pour le fichier {json_path}")
                        continue
                    
                    image = plt.imread(image_path)
                    height, width = image.shape[:2]

                    self.add_image(
                        "flower",
                        image_id=filename,
                        path=image_path,
                        width=width, height=height,
                        polygons=polygons,
                        class_id=label_id
                    )

    def load_mask(self, image_id):
        image_info = self.image_info[image_id]
        if image_info["source"] != "flower":
            return super(self.__class__, self).load_mask(image_id)
        info = self.image_info[image_id]
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])], dtype=np.uint8)
        class_ids = np.array([info["class_id"]] * len(info["polygons"]))
        for i, polygon in enumerate(info["polygons"]):
            polygon = np.array(polygon, dtype=int)  # Convertir en tableau NumPy entier
            rr, cc = polygon[:, 1].astype(int), polygon[:, 0].astype(int)
            mask[rr, cc, i] = 1
        return mask.astype(np.bool), class_ids

# Charger le dataset
dataset_dir = 'data_t'
train_dataset = FlowerDataset()
train_dataset.load_flowers(dataset_dir, "train")
train_dataset.prepare()

val_dataset = FlowerDataset()
val_dataset.load_flowers(dataset_dir, "val")
val_dataset.prepare()

# Initialiser le modèle
model = MaskRCNN(mode="training", config=FlowerConfig(), model_dir='./logs')
model.load_weights('mask_rcnn_coco.h5', by_name=True, exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", "mrcnn_bbox", "mrcnn_mask"])

# Entraîner le modèle
augmentation = imgaug.augmenters.Fliplr(0.5)
model.train(train_dataset, val_dataset, learning_rate=FlowerConfig().LEARNING_RATE, epochs=20, layers='heads', augmentation=augmentation)